# Programmatic access to the ASQ platform

This notebook shows how to use the ASQ API for programmatic access to cover the key elements in a corresponding interactive session.

We assume users at least have been familiar with how to use the [interactive session](https://asq.epigraphdb.org) and will not cover conceptual details here. 
- Technical specifications related to the API can be found at [its Swagger UI](https://asq-api.epigraphdb.org).
  - Including configurable parameters, which we will use default settings to keep things simple.
- General documentation related to the ASQ platform and research can be found at [ASQ's documentation page](https://asq.epigraphdb.org/docs).
- Feedback and questions are very welcome -- please reach out via [Github](https://github.com/mrcieu/epigraphdb-asq/issues).

In [35]:
# The following dependency should be installed
from typing import List
from typing_extensions import TypedDict

import requests
import pandas as pd
from pydash import py_

In [3]:
API_URL = "https://asq-api.epigraphdb.org"

In [73]:
class Ent(TypedDict):
    ent_id: str
    ent_term: str
    
class TripleItem(TypedDict):
    triple_id: str
    triple_label: str
    
class AssocEvidenceScoreInput(TypedDict):
    idx: int
    subject_id: str
    subject_term: str
    object_id: str
    object_term: str
    effect_size: float
    se: float
    pval: float
    
class TripleLiteratureEvidenceScoreInput(TypedDict):
    idx: int
    ent_subject_id: str
    ent_subject_term: str
    ent_object_id: str
    ent_object_term: str
    literature_count: int
    
class EntMapping(TypedDict):
    ent_id: str
    ent_term: str
    ref_ent_id: str
    ref_ent_term: str
    similarity_score: float
    
class OntologyEntMapping(TypedDict):
    identity_score: float
    ic_score: float
    
class UmlsEntMapping(TypedDict):
    ref_meta_ent: str
    identity_score: float
    ic_score: float

# Parsing claims from text, and select a triple

Here we used one example text (Dixon and Peters, 2018) to showcase the functionalities.

In [7]:
CLAIM_TEXT = """\
There is a major epidemic of obesity, and many obese patients suffer with respiratory symptoms and disease.
The overall impact of obesity on lung function is multifactorial, related to mechanical and inflammatory aspects of obesity.
Areas covered: Obesity causes substantial changes to the mechanics of the lungs and chest wall, and these mechanical changes cause asthma and asthma-like symptoms such as dyspnea, wheeze, and airway hyperresponsiveness.
Excess adiposity is also associated with increased production of inflammatory cytokines and immune cells that may also lead to disease.
This article reviews the literature addressing the relationship between obesity and lung function, and studies addressing how the mechanical and inflammatory effects of obesity might lead to changes in lung mechanics and pulmonary function in obese adults and children.
Expert commentary: Obesity has significant effects on respiratory function, which contribute significantly to the burden of respiratory disease.
These mechanical effects are not readily quantified with conventional pulmonary function testing and measurement of body mass index.
Changes in mediators produced by adipose tissue likely also contribute to altered lung function, though as of yet this is poorly understood.
"""
print(CLAIM_TEXT)

There is a major epidemic of obesity, and many obese patients suffer with respiratory symptoms and disease.
The overall impact of obesity on lung function is multifactorial, related to mechanical and inflammatory aspects of obesity.
Areas covered: Obesity causes substantial changes to the mechanics of the lungs and chest wall, and these mechanical changes cause asthma and asthma-like symptoms such as dyspnea, wheeze, and airway hyperresponsiveness.
Excess adiposity is also associated with increased production of inflammatory cytokines and immune cells that may also lead to disease.
This article reviews the literature addressing the relationship between obesity and lung function, and studies addressing how the mechanical and inflammatory effects of obesity might lead to changes in lung mechanics and pulmonary function in obese adults and children.
Expert commentary: Obesity has significant effects on respiratory function, which contribute significantly to the burden of respiratory disea

In [8]:
r = requests.post(API_URL + "/claim_parsing/parse", json={"claim_text": CLAIM_TEXT})
r.raise_for_status()
parsed_results = r.json()
print(parsed_results.keys())

dict_keys(['data', 'html', 'invalid_triples', 'claim_text'])


In [9]:
# Triples that are not supported, i.e. invalid_triples
pd.json_normalize(parsed_results["invalid_triples"])

,idx,sub_id,sub_term,sub_type,sub_text,sub_neg,sub_confidence_score,sub_start_pos,sub_end_pos,pred_type,...,obj_id,obj_term,obj_type,obj_text,obj_neg,obj_confidence_score,obj_start_pos,obj_end_pos,text,triple_text
0,0,C0028754,Obesity,dsyn,obese,False,888,47,52,MOD/HEAD,...,C0030705,Patients,humn,patients,False,888,53,61,"There is a major epidemic of obesity, and many...",Obesity:PROCESS_OF:Patients
1,1,C0012634,Disease,dsyn,disease,False,1000,99,106,VERB,...,C0030705,Patients,humn,patients,False,1000,53,61,"There is a major epidemic of obesity, and many...",Disease:PROCESS_OF:Patients
2,2,C0037090,"Signs and Symptoms, Respiratory",sosy,respiratory symptoms,False,1000,74,94,VERB,...,C0030705,Patients,humn,patients,False,1000,53,61,"There is a major epidemic of obesity, and many...","Signs and Symptoms, Respiratory:PROCESS_OF:Pat..."
3,3,C0013404,Dyspnea,sosy,dyspnea,False,1000,171,178,SPEC,...,C1457887,Symptoms,sosy,symptoms,False,1000,154,162,Areas covered: Obesity causes substantial chan...,Dyspnea:ISA:Symptoms
4,4,C0035228,Respiratory Hypersensitivity,patf,airway hyperresponsiveness,False,1000,192,218,SPEC,...,C1457887,Symptoms,sosy,symptoms,False,1000,154,162,Areas covered: Obesity causes substantial chan...,Respiratory Hypersensitivity:ISA:Symptoms
5,5,C0043144,Wheezing,sosy,wheeze,False,1000,180,186,SPEC,...,C1457887,Symptoms,sosy,symptoms,False,1000,154,162,Areas covered: Obesity causes substantial chan...,Wheezing:ISA:Symptoms
6,6,C0028754,Obesity,dsyn,obese,False,888,243,248,MOD/HEAD,...,C0001675,Adult,humn,adults,False,888,249,255,This article reviews the literature addressing...,Obesity:PROCESS_OF:Adult
7,7,C0231921,Pulmonary function,ortf,pulmonary function,False,1000,221,239,PREP,...,C0001675,Adult,humn,adults,False,1000,249,255,This article reviews the literature addressing...,Pulmonary function:PROCESS_OF:Adult
8,8,C0231921,Pulmonary function,ortf,pulmonary function,False,1000,221,239,PREP,...,C0008059,Child,aggp,children,False,1000,260,268,This article reviews the literature addressing...,Pulmonary function:PROCESS_OF:Child


In [10]:
# Triples that are supported
pd.json_normalize(parsed_results["data"])

,idx,sub_id,sub_term,sub_type,sub_text,sub_neg,sub_confidence_score,sub_start_pos,sub_end_pos,pred_type,...,obj_id,obj_term,obj_type,obj_text,obj_neg,obj_confidence_score,obj_start_pos,obj_end_pos,text,triple_text
0,0,C0028754,Obesity,dsyn,obesity,False,1000,22,29,NOM,...,C0035245,Respiratory physiology,phsf,lung function,False,1000,33,46,The overall impact of obesity on lung function...,Obesity:AFFECTS:Respiratory physiology
1,1,C0028754,Obesity,dsyn,Obesity,False,1000,15,22,VERB,...,C0004096,Asthma,dsyn,asthma,False,1000,131,137,Areas covered: Obesity causes substantial chan...,Obesity:CAUSES:Asthma
2,2,C0028754,Obesity,dsyn,Obesity,False,1000,15,22,VERB,...,C1457887,Symptoms,sosy,symptoms,False,1000,154,162,Areas covered: Obesity causes substantial chan...,Obesity:CAUSES:Symptoms
3,3,C0028754,Obesity,dsyn,adiposity,False,888,7,16,VERB,...,C0012634,Disease,dsyn,disease,False,888,127,134,Excess adiposity is also associated with incre...,Obesity:CAUSES:Disease
4,4,C0035245,Respiratory physiology,phsf,lung function,False,1000,84,97,NOM,...,C0028754,Obesity,dsyn,obesity,False,1000,169,176,This article reviews the literature addressing...,Respiratory physiology:AFFECTS:Obesity
5,5,C0028754,Obesity,dsyn,Obesity,False,1000,19,26,NOM,...,C0035203,Respiration,phsf,respiratory function,False,1000,54,74,Expert commentary: Obesity has significant eff...,Obesity:AFFECTS:Respiration


We then select one triple of interest for further analysis

In [11]:
IDX = 1

In [12]:
claim_triple = parsed_results["data"][IDX]
print(claim_triple)

{'idx': 1, 'sub_id': 'C0028754', 'sub_term': 'Obesity', 'sub_type': 'dsyn', 'sub_text': 'Obesity', 'sub_neg': False, 'sub_confidence_score': 1000, 'sub_start_pos': 15, 'sub_end_pos': 22, 'pred_type': 'VERB', 'pred': 'CAUSES', 'pred_start_pos': 125, 'pred_end_pos': 130, 'obj_id': 'C0004096', 'obj_term': 'Asthma', 'obj_type': 'dsyn', 'obj_text': 'asthma', 'obj_neg': False, 'obj_confidence_score': 1000, 'obj_start_pos': 131, 'obj_end_pos': 137, 'text': 'Areas covered: Obesity causes substantial changes to the mechanics of the lungs and chest wall, and these mechanical changes cause asthma and asthma-like symptoms such as dyspnea, wheeze, and airway hyperresponsiveness.', 'triple_text': 'Obesity:CAUSES:Asthma'}


# Entity harmonization stage

In [13]:
claim_ents = {
    "subject": {
        "ent_id": claim_triple["sub_id"],
        "ent_term": claim_triple["sub_term"],
    },
    "object": {
        "ent_id": claim_triple["obj_id"],
        "ent_term": claim_triple["obj_term"],
    },
}
print(claim_ents)

{'subject': {'ent_id': 'C0028754', 'ent_term': 'Obesity'}, 'object': {'ent_id': 'C0004096', 'ent_term': 'Asthma'}}


## From claim entities to ontology entities

In [21]:
def map_claim_ent_to_ontology_ent(ent_id: str, ent_term: str):
    url = API_URL + "/ent_harmonization/ontology_ents"
    payload = {"ent_id": ent_id, "ent_term": ent_term}
    r = requests.post(url, json=payload)
    r.raise_for_status()
    r_res = r.json()
    return r_res
ontology_ent_res = (
    py_
    .chain(claim_ents)
    .map_values(
        lambda v: 
        map_claim_ent_to_ontology_ent(ent_id=v["ent_id"], ent_term=v["ent_term"]))
    .value()
)

In [23]:
# this is the structure of the return data
for k, v in ontology_ent_res.items():
    print(k, v.keys())

subject dict_keys(['candidates', 'ents'])
object dict_keys(['candidates', 'ents'])


In [28]:
# Here are what has been retrieved as candidates (selected + not select)
print("Candidates")
(
    py_
    .chain(ontology_ent_res)
    .map(lambda v, k: pd.json_normalize(v["candidates"]).assign(ent_type=k))
    .reduce(
        lambda a, b: pd.concat([a, b]).reset_index(drop=True)
    )
    .value()
)

Candidates


,ent_id,ent_term,similarity_score,identity_score,ic_score,url,ent_type
0,http://www.orpha.net/ORDO/Orphanet_240371,Syndromic obesity,0.773042,1.914597,0.888326,https://epigraphdb.org/entity?meta_node=Efo&id...,subject
1,http://www.ebi.ac.uk/efo/EFO_0001073,obesity,1.000000,0.692054,0.810444,https://epigraphdb.org/entity?meta_node=Efo&id...,subject
2,http://www.ebi.ac.uk/efo/EFO_0001074,morbid obesity,0.859996,1.901786,0.991864,https://epigraphdb.org/entity?meta_node=Efo&id...,subject
3,http://www.ebi.ac.uk/efo/EFO_0003774,hyperlipidemia,0.705085,11.135805,0.977317,https://epigraphdb.org/entity?meta_node=Efo&id...,subject
4,http://purl.obolibrary.org/obo/HP_0001956,Truncal obesity,0.796701,1.917169,0.991864,https://epigraphdb.org/entity?meta_node=Efo&id...,subject
5,http://www.orpha.net/ORDO/Orphanet_77828,Genetic obesity,0.836022,0.581927,0.870050,https://epigraphdb.org/entity?meta_node=Efo&id...,subject
6,http://www.orpha.net/ORDO/Orphanet_179490,Obesity due to congenital leptin resistance,0.702334,2.368193,0.984331,https://epigraphdb.org/entity?meta_node=Efo&id...,subject
7,http://www.ebi.ac.uk/efo/EFO_0000274,atopic eczema,0.757739,6.258821,0.970757,https://epigraphdb.org/entity?meta_node=Efo&id...,object
8,http://www.ebi.ac.uk/efo/EFO_0003956,seasonal allergic rhinitis,0.791270,5.650232,0.995271,https://epigraphdb.org/entity?meta_node=Efo&id...,object
9,http://www.ebi.ac.uk/efo/EFO_0005854,allergic rhinitis,0.820938,5.966584,0.985793,https://epigraphdb.org/entity?meta_node=Efo&id...,object


In [29]:
# Here are what has been retrieved as entities (selected)
print("Entities")
(
    py_
    .chain(ontology_ent_res)
    .map(lambda v, k: pd.json_normalize(v["ents"]).assign(ent_type=k))
    .reduce(
        lambda a, b: pd.concat([a, b]).reset_index(drop=True)
    )
    .value()
)

Entities


,ent_id,ent_term,similarity_score,identity_score,ic_score,url,ent_type
0,http://www.ebi.ac.uk/efo/EFO_0001073,obesity,1.000000,0.692054,0.810444,https://epigraphdb.org/entity?meta_node=Efo&id...,subject
1,http://www.orpha.net/ORDO/Orphanet_77828,Genetic obesity,0.836022,0.581927,0.870050,https://epigraphdb.org/entity?meta_node=Efo&id...,subject
2,http://www.ebi.ac.uk/efo/EFO_0000270,asthma,1.000000,0.521951,0.968473,https://epigraphdb.org/entity?meta_node=Efo&id...,object


In [31]:
ontology_ents = (
    py_
    .chain(ontology_ent_res)
    .map_values(
        lambda v: 
        [
            {"ent_id": _["ent_id"], "ent_term": _["ent_term"]} 
            for _ in v["ents"]
        ]
    )
    .value()
)
print(ontology_ents)

{'subject': [{'ent_id': 'http://www.ebi.ac.uk/efo/EFO_0001073', 'ent_term': 'obesity'}, {'ent_id': 'http://www.orpha.net/ORDO/Orphanet_77828', 'ent_term': 'Genetic obesity'}], 'object': [{'ent_id': 'http://www.ebi.ac.uk/efo/EFO_0000270', 'ent_term': 'asthma'}]}


## From ontology entities to evidence entities

In [37]:
def map_ontology_ent_to_umls_ent(query_ent: Ent, ontology_ents: List[Ent]):
    url = API_URL + "/ent_harmonization/umls_ents"
    payload = {
        "query_umls_ent": query_ent,
        "ontology_ents": ontology_ents,
    }
    r = requests.post(url, json=payload)
    r.raise_for_status()
    r_res = r.json()
    return r_res
umls_ent_res = (
    py_
    .chain(ontology_ents)
    .map_values(
        lambda v, k:
          map_ontology_ent_to_umls_ent(query_ent=claim_ents[k], ontology_ents=v)
    )
    .value()
)
for k, v in umls_ent_res.items():
    print(k, v.keys())

subject dict_keys(['ents', 'detail_data'])
object dict_keys(['ents', 'detail_data'])


In [39]:
def map_ontology_ent_to_trait_ent(ontology_ents: List[Ent], pred_term: str):
    url = API_URL + "/ent_harmonization/trait_ents"
    payload = {
        "ents": ontology_ents,
        "pred_term": pred_term
    }
    r = requests.post(url, json=payload)
    r.raise_for_status()
    r_res = r.json()
    return r_res
trait_ent_res = (
    py_
    .chain(ontology_ents)
    .map_values(
        lambda v:
          map_ontology_ent_to_trait_ent(ontology_ents=v, pred_term=claim_triple["pred"])
    )
    .value()
)
for k, v in trait_ent_res.items():
    print(k, v.keys())

subject dict_keys(['ents', 'detail_data'])
object dict_keys(['ents', 'detail_data'])


Here are what has been retrieved:

In [44]:
evidence_ent_res = py_.chain([
    py_
    .chain(trait_ent_res)
    .map(
        lambda v, k:
          pd.json_normalize(v["detail_data"]).assign(ent_group="Trait", ent_type=k)
    )
    .value(),
    py_
    .chain(umls_ent_res)
    .map(
        lambda v, k:
          pd.json_normalize(v["detail_data"]).assign(ent_group="UMLS", ent_type=k)
    )
    .value(),
]).flatten().thru(lambda o: pd.concat(o).reset_index(drop=True)).value()
evidence_ent_res

,ent_id,ent_term,ent_url,meta_ent,meta_ent_url,ref_ent_id,ref_ent_term,ref_ent_url,ref_meta_ent,ref_meta_ent_url,similarity_score,ent_group,ent_type
0,ieu-a-1096,Childhood obesity,https://epigraphdb.org/entity?meta_node=Gwas&i...,Gwas,https://epigraphdb.org/meta-node/Gwas,http://www.ebi.ac.uk/efo/EFO_0001073,obesity,https://epigraphdb.org/entity?meta_node=Efo&id...,Efo,https://epigraphdb.org/meta-node/Efo,0.877615,Trait,subject
1,ieu-a-93,Overweight,https://epigraphdb.org/entity?meta_node=Gwas&i...,Gwas,https://epigraphdb.org/meta-node/Gwas,http://www.ebi.ac.uk/efo/EFO_0001073,obesity,https://epigraphdb.org/entity?meta_node=Efo&id...,Efo,https://epigraphdb.org/meta-node/Efo,0.714371,Trait,subject
2,ieu-a-1096,Childhood obesity,https://epigraphdb.org/entity?meta_node=Gwas&i...,Gwas,https://epigraphdb.org/meta-node/Gwas,http://www.orpha.net/ORDO/Orphanet_77828,Genetic obesity,https://epigraphdb.org/entity?meta_node=Efo&id...,Efo,https://epigraphdb.org/meta-node/Efo,0.749311,Trait,subject
3,ukb-d-J10_ASTHMA,Asthma,https://epigraphdb.org/entity?meta_node=Gwas&i...,Gwas,https://epigraphdb.org/meta-node/Gwas,http://www.ebi.ac.uk/efo/EFO_0000270,asthma,https://epigraphdb.org/entity?meta_node=Efo&id...,Efo,https://epigraphdb.org/meta-node/Efo,1.000000,Trait,object
4,ieu-a-44,Asthma,https://epigraphdb.org/entity?meta_node=Gwas&i...,Gwas,https://epigraphdb.org/meta-node/Gwas,http://www.ebi.ac.uk/efo/EFO_0000270,asthma,https://epigraphdb.org/entity?meta_node=Efo&id...,Efo,https://epigraphdb.org/meta-node/Efo,1.000000,Trait,object
...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,C1260881,Allergic bronchitis,https://epigraphdb.org/entity?meta_node=Litera...,LiteratureTerm,https://epigraphdb.org/meta-node/LiteratureTerm,http://www.ebi.ac.uk/efo/EFO_0000270,asthma,https://epigraphdb.org/entity?meta_node=Efo&id...,Efo,https://epigraphdb.org/meta-node/Efo,0.819931,UMLS,object
98,C0264348,Chronic asthmatic bronchitis,https://epigraphdb.org/entity?meta_node=Litera...,LiteratureTerm,https://epigraphdb.org/meta-node/LiteratureTerm,http://www.ebi.ac.uk/efo/EFO_0000270,asthma,https://epigraphdb.org/entity?meta_node=Efo&id...,Efo,https://epigraphdb.org/meta-node/Efo,0.803360,UMLS,object
99,C0861154,Allergic rhinoconjunctivitis,https://epigraphdb.org/entity?meta_node=Litera...,LiteratureTerm,https://epigraphdb.org/meta-node/LiteratureTerm,http://www.ebi.ac.uk/efo/EFO_0000270,asthma,https://epigraphdb.org/entity?meta_node=Efo&id...,Efo,https://epigraphdb.org/meta-node/Efo,0.803044,UMLS,object
100,C0684913,Chemical-induced asthma,https://epigraphdb.org/entity?meta_node=Litera...,LiteratureTerm,https://epigraphdb.org/meta-node/LiteratureTerm,http://www.ebi.ac.uk/efo/EFO_0000270,asthma,https://epigraphdb.org/entity?meta_node=Efo&id...,Efo,https://epigraphdb.org/meta-node/Efo,0.800429,UMLS,object


# Evidence retrieval stage

To keep things simple we will only cover supporting (`supporting`) evidence here. Users who are interested in further details should refer to the test cases in the code repository.

In [46]:
EVIDENCE_TYPE = "supporting"

## Retrieving evidence

In [50]:
def get_triple_literature_evidence(
    subject_ents: List[Ent], 
    object_ents: List[Ent], 
    pred_term: str, 
    evidence_type: str,
):
    url = API_URL + "/evidence/triples"
    payload = {
        "evidence_type": evidence_type,
        "subject_ents": subject_ents,
        "object_ents": object_ents,
        "pred_term": pred_term
    }
    r = requests.post(url, json=payload)
    r.raise_for_status()
    r_res = r.json()
    return r_res

triple_literature_evidence = get_triple_literature_evidence(
    subject_ents=umls_ent_res["subject"]["ents"],
    object_ents=umls_ent_res["object"]["ents"],
    pred_term=claim_triple["pred"],
    evidence_type=EVIDENCE_TYPE
)

In [52]:
pd.json_normalize(triple_literature_evidence)

,triple_id,triple_label,triple_lower,triple_subject_id,triple_subject,triple_object_id,triple_object,triple_predicate,ent_subject_id,ent_object_id,ent_subject_term,ent_object_term,direction,literature_count
0,C0028754:CAUSES:C0004096,Obesity CAUSES Asthma,obesity:causes:asthma,C0028754,Obesity,C0004096,Asthma,CAUSES,C0028754,C0004096,Obesity,Asthma,forward,20
1,C0242339:CAUSES:C0004096,Dyslipidemias CAUSES Asthma,dyslipidemias:causes:asthma,C0242339,Dyslipidemias,C0004096,Asthma,CAUSES,C0242339,C0004096,Dyslipidemias,Asthma,forward,1


In [57]:
def get_literature_details(
    triple_items: List[TripleItem],
):
    url = API_URL + "/evidence/literature"
    payload = {
        "triple_items": triple_items
    }
    r = requests.post(url, json=payload)
    r.raise_for_status()
    r_res = r.json()
    return r_res

    
# Just get details for one triple item, they can be really slow
literature_details = get_literature_details(
    triple_items=[
        {
            "triple_id": triple_literature_evidence[0]["triple_id"],
            "triple_label": triple_literature_evidence[0]["triple_lower"],
        }
    ]
)

In [58]:
pd.json_normalize(literature_details)

,data,html_text
0,[{'abstract': 'BACKGROUND: Asthma in obese sub...,"[{'idx': 0, 'title_text': '<div class=""tex2jax..."


In [63]:
def get_assoc_evidence(
    subject_ents: List[Ent], 
    object_ents: List[Ent], 
    pred_term: str, 
    evidence_type: str,
):
    url = API_URL + "/evidence/association"
    payload = {
        "evidence_type": evidence_type,
        "subject_ents": subject_ents,
        "object_ents": object_ents,
        "pred_term": pred_term
    }
    r = requests.post(url, json=payload)
    r.raise_for_status()
    r_res = r.json()["data"]
    return r_res

assoc_evidence = get_assoc_evidence(
    subject_ents=trait_ent_res["subject"]["ents"],
    object_ents=trait_ent_res["object"]["ents"],
    pred_term=claim_triple["pred"],
    evidence_type=EVIDENCE_TYPE
)

In [64]:
pd.json_normalize(assoc_evidence)

,subject_id,subject_term,object_id,object_term,meta_rel,direction,effect_size,se,pval,rel_data.b,rel_data.se,rel_data.method,rel_data.nsnp,rel_data.selection,rel_data.pval,rel_data.ci_upp,rel_data._source,rel_data.ci_low,rel_data.moescore
0,ieu-a-1096,Childhood obesity,ukb-b-12753,Recent medication for hayfever or allergic rhi...,MR_EVE_MR,forward,-0.023757,0.007484,0.001502,-0.023757,0.007484,FE IVW,5,DF,0.001502,-0.009088,[MR-EvE-2021-03-10],-0.038426,1.00
1,ieu-a-1096,Childhood obesity,ukb-a-446,Blood clot DVT bronchitis emphysema asthma...,MR_EVE_MR,forward,0.004130,0.001351,0.002231,0.004130,0.001351,FE IVW,5,DF,0.002231,0.006777,[MR-EvE-2021-03-10],0.001483,1.00
2,ieu-a-93,Overweight,ukb-a-446,Blood clot DVT bronchitis emphysema asthma...,MR_EVE_MR,forward,0.012148,0.002924,0.000033,0.012148,0.002924,FE IVW,14,DF,0.000033,0.017943,[MR-EvE-2021-03-10],0.006353,0.79


## Assign scores

In [67]:
def get_assoc_scores(
    assoc_evidence: List[AssocEvidenceScoreInput],
    query_subject_term: str,
    query_object_term: str,
    ontology_subject_mapping: List[OntologyEntMapping],
    ontology_object_mapping: List[OntologyEntMapping],
    trait_subject_mapping: List[EntMapping],
    trait_object_mapping: List[EntMapping],
):
    url = API_URL + "/scores/assoc"
    payload = {
        "assoc_evidence": assoc_evidence,
        "query_subject_term": query_subject_term,
        "query_object_term": query_object_term,
        "ontology_subject_mapping": ontology_subject_mapping,
        "ontology_object_mapping": ontology_object_mapping,
        "trait_subject_mapping": trait_subject_mapping,
        "trait_object_mapping": trait_object_mapping,
    }
    r = requests.post(url, json=payload)
    r.raise_for_status()
    r_res = r.json()["data"]
    return r_res

assoc_evidence_scores = get_assoc_scores(
    assoc_evidence=pd.json_normalize(assoc_evidence)[[
        "subject_id", "subject_term", "object_id", "object_term", 
        "effect_size", "se", "pval"]]
        .reset_index(drop=False)
        .rename(columns={"index": "idx"})
        .to_dict(orient="records"),
    query_subject_term=claim_ents["subject"]["ent_term"],
    query_object_term=claim_ents["subject"]["ent_term"],
    ontology_subject_mapping=pd.json_normalize(ontology_ent_res["subject"]["ents"])[[
        "ent_id", "ent_term", "similarity_score", "identity_score", "ic_score"]]
        .assign(
            ref_ent_id=claim_ents["subject"]["ent_id"],
            ref_ent_term=claim_ents["subject"]["ent_term"],
        )
        .to_dict(orient="records"),
    ontology_object_mapping=pd.json_normalize(ontology_ent_res["object"]["ents"])[[
        "ent_id", "ent_term", "similarity_score", "identity_score", "ic_score"]]
        .assign(
            ref_ent_id=claim_ents["object"]["ent_id"],
            ref_ent_term=claim_ents["object"]["ent_term"],
        )
        .to_dict(orient="records"),
    trait_subject_mapping=pd.json_normalize(trait_ent_res["subject"]["detail_data"])[[
        "ent_id", "ent_term", "ref_ent_id", "ref_ent_term", "similarity_score"]]
        .to_dict(orient="records"),
    trait_object_mapping=pd.json_normalize(trait_ent_res["object"]["detail_data"])[[
        "ent_id", "ent_term", "ref_ent_id", "ref_ent_term", "similarity_score"]]
        .to_dict(orient="records"),
)

In [69]:
pd.json_normalize(assoc_evidence_scores)

,idx,subject_id,subject_term,object_id,object_term,effect_size,se,pval,mapping_score,assoc_score,evidence_score,mapping_data.subject_mapping_score,mapping_data.object_mapping_score,mapping_data.subject_mapping_data,mapping_data.object_mapping_data
0,0,ieu-a-1096,Childhood obesity,ukb-b-12753,Recent medication for hayfever or allergic rhi...,-0.023757,0.007484,0.001502,0.738575,1.501648,1.109080,0.877615,0.841571,"[{'ent_id': 'ieu-a-1096', 'ent_term': 'Childho...","[{'ent_id': 'ukb-b-12753', 'ent_term': 'Recent..."
1,1,ieu-a-1096,Childhood obesity,ukb-a-446,Blood clot DVT bronchitis emphysema asthma...,0.004130,0.001351,0.002231,0.754852,1.485390,1.121250,0.877615,0.860118,"[{'ent_id': 'ieu-a-1096', 'ent_term': 'Childho...","[{'ent_id': 'ukb-a-446', 'ent_term': 'Blood cl..."
2,2,ieu-a-93,Overweight,ukb-a-446,Blood clot DVT bronchitis emphysema asthma...,0.012148,0.002924,0.000033,0.614443,1.618455,0.994448,0.714371,0.860118,"[{'ent_id': 'ieu-a-93', 'ent_term': 'Overweigh...","[{'ent_id': 'ukb-a-446', 'ent_term': 'Blood cl..."


In [74]:
def get_triple_literature_scores(
    triple_literature_evidence: List[TripleLiteratureEvidenceScoreInput],
    query_subject_term: str,
    query_object_term: str,
    ontology_subject_mapping: List[OntologyEntMapping],
    ontology_object_mapping: List[OntologyEntMapping],
    umls_subject_mapping: List[UmlsEntMapping],
    umls_object_mapping: List[UmlsEntMapping],
):
    url = API_URL + "/scores/triples"
    payload = {
        "triple_evidence": triple_literature_evidence,
        "query_subject_term": query_subject_term,
        "query_object_term": query_object_term,
        "ontology_subject_mapping": ontology_subject_mapping,
        "ontology_object_mapping": ontology_object_mapping,
        "umls_subject_mapping": umls_subject_mapping,
        "umls_object_mapping": umls_object_mapping,
    }
    r = requests.post(url, json=payload)
    r.raise_for_status()
    r_res = r.json()["data"]
    return r_res

triple_literature_evidence_scores = get_triple_literature_scores(
    triple_literature_evidence=pd.json_normalize(triple_literature_evidence)
        .reset_index(drop=False)
        .rename(columns={
            "index": "idx",
            "triple_subject_id": "ent_subject_id",
            "triple_subject_term": "ent_subject_term",
            "triple_object_id": "ent_object_id",
            "triple_object_term": "ent_object_term",
        })
        .to_dict(orient="records"),
    query_subject_term=claim_ents["subject"]["ent_term"],
    query_object_term=claim_ents["subject"]["ent_term"],
    ontology_subject_mapping=pd.json_normalize(ontology_ent_res["subject"]["ents"])[[
        "ent_id", "ent_term", "similarity_score", "identity_score", "ic_score"]]
        .assign(
            ref_ent_id=claim_ents["subject"]["ent_id"],
            ref_ent_term=claim_ents["subject"]["ent_term"],
        )
        .to_dict(orient="records"),
    ontology_object_mapping=pd.json_normalize(ontology_ent_res["object"]["ents"])[[
        "ent_id", "ent_term", "similarity_score", "identity_score", "ic_score"]]
        .assign(
            ref_ent_id=claim_ents["object"]["ent_id"],
            ref_ent_term=claim_ents["object"]["ent_term"],
        )
        .to_dict(orient="records"),
    umls_subject_mapping=pd.json_normalize(umls_ent_res["subject"]["detail_data"])[[
        "ent_id", "ent_term", "ref_meta_ent", "ref_ent_id", "ref_ent_term", "similarity_score"]]
        .to_dict(orient="records"),
    umls_object_mapping=pd.json_normalize(umls_ent_res["object"]["detail_data"])[[
        "ent_id", "ent_term", "ref_meta_ent", "ref_ent_id", "ref_ent_term", "similarity_score"]]
        .to_dict(orient="records"),
)

/tmp/ipykernel_32564/4235533242.py:26: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  triple_literature_evidence=pd.json_normalize(triple_literature_evidence)


In [75]:
pd.json_normalize(triple_literature_evidence_scores)

,idx,ent_subject_id,ent_subject_term,ent_object_id,ent_object_term,literature_count,mapping_score,triple_score,evidence_score,mapping_data.subject_mapping_score,mapping_data.object_mapping_score,mapping_data.subject_mapping_data,mapping_data.object_mapping_data
0,0,C0028754,Obesity,C0004096,Asthma,20,1.000000,2.30103,2.301030,1.000000,1.0,"[{'ent_id': 'C0028754', 'ent_term': 'Obesity',...","[{'ent_id': 'C0004096', 'ent_term': 'Asthma', ..."
1,1,C0242339,Dyslipidemias,C0004096,Asthma,1,0.775928,1.00000,0.775928,0.775928,1.0,"[{'ent_id': 'C0242339', 'ent_term': 'Dyslipide...","[{'ent_id': 'C0004096', 'ent_term': 'Asthma', ..."


Here concludes the tutorial, where we showed the key elements regarding the programmatic access to ASQ platform.

We assume users at least have been familiar with how to use the [interactive session](https://asq.epigraphdb.org) and will not cover conceptual details here. 
- Technical specifications related to the API can be found at [its Swagger UI](https://asq-api.epigraphdb.org).
  - Including configurable parameters, which we will use default settings to keep things simple.
- General documentation related to the ASQ platform and research can be found at [ASQ's documentation page](https://asq.epigraphdb.org/docs).
- Feedback and questions are very welcome -- please reach out via [Github](https://github.com/mrcieu/epigraphdb-asq/issues).

# Reference

- Dixon AE, Peters U. The effect of obesity on lung function. Expert Rev Respir Med. 2018 Sep;12(9):755-767. doi: 10.1080/17476348.2018.1506331. Epub 2018 Aug 14. PMID: 30056777; PMCID: PMC6311385.